In [1]:
import numpy as np 
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

In [2]:
df = pd.read_csv('/kaggle/input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv')
df
classes = sorted(df.emotion)

In [3]:
df

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


In [4]:
counter = 0
for i in range(len(df)):
    if df.iloc[i]['Usage'] == 'Training':
        counter +=1

print(counter)

28709


# **Data Structuring**

In [5]:
dataset = []
for label in range(len(df)):
    element = df.iloc[label]['pixels'],df[df.pixels==df.iloc[label]['pixels']].iloc[0]['emotion']
    tf = np.array(element[0].split(' ')).astype('float32').reshape(1, 48, 48)
    tf = torch.from_numpy(tf)
    tf = [tf, torch.tensor(element[1])]
    dataset.append(tf)
    #0:anger, 1:disgust, 2:fear 3:happiness, 4:sadness, 5:suprise, 6:neutral

    

In [6]:
dataset

[[tensor([[[ 70.,  80.,  82.,  ...,  52.,  43.,  41.],
           [ 65.,  61.,  58.,  ...,  56.,  52.,  44.],
           [ 50.,  43.,  54.,  ...,  49.,  56.,  47.],
           ...,
           [ 91.,  65.,  42.,  ...,  72.,  56.,  43.],
           [ 77.,  82.,  79.,  ..., 105.,  70.,  46.],
           [ 77.,  72.,  84.,  ..., 106., 109.,  82.]]]),
  tensor(0)],
 [tensor([[[151., 150., 147.,  ..., 129., 140., 120.],
           [151., 149., 149.,  ..., 122., 141., 137.],
           [151., 151., 156.,  ..., 109., 123., 146.],
           ...,
           [188., 188., 121.,  ..., 185., 185., 186.],
           [188., 187., 196.,  ..., 186., 182., 187.],
           [186., 184., 185.,  ..., 193., 183., 184.]]]),
  tensor(0)],
 [tensor([[[231., 212., 156.,  ...,  44.,  27.,  16.],
           [229., 175., 148.,  ...,  27.,  35.,  27.],
           [214., 156., 157.,  ...,  28.,  22.,  28.],
           ...,
           [241., 245., 250.,  ...,  57., 101., 146.],
           [246., 250., 252.,  ...,  7

In [7]:
train_set = dataset[:28709]
test_set = dataset[28709:]
dataset[0][1]

tensor(0)

In [8]:
def dataloader(data_set, batch_size):
    trainset = []
    size = len(data_set)
    for index in range(0, size, batch_size):
        img_data = []
        label_data = []
       
        for datapoint in data_set[index:index+batch_size]:
            X, Y = datapoint
            img_data.append(X)
            label_data.append(Y.item())    
            
        datapoint = [torch.cat(img_data), torch.tensor(label_data)]
        trainset.append(datapoint)
            
    return trainset
        
        
        

In [9]:
trainset= dataloader(train_set, 7)
testset = dataloader(train_set, 7)


In [10]:
trainset

[[tensor([[[ 70.,  80.,  82.,  ...,  52.,  43.,  41.],
           [ 65.,  61.,  58.,  ...,  56.,  52.,  44.],
           [ 50.,  43.,  54.,  ...,  49.,  56.,  47.],
           ...,
           [ 91.,  65.,  42.,  ...,  72.,  56.,  43.],
           [ 77.,  82.,  79.,  ..., 105.,  70.,  46.],
           [ 77.,  72.,  84.,  ..., 106., 109.,  82.]],
  
          [[151., 150., 147.,  ..., 129., 140., 120.],
           [151., 149., 149.,  ..., 122., 141., 137.],
           [151., 151., 156.,  ..., 109., 123., 146.],
           ...,
           [188., 188., 121.,  ..., 185., 185., 186.],
           [188., 187., 196.,  ..., 186., 182., 187.],
           [186., 184., 185.,  ..., 193., 183., 184.]],
  
          [[231., 212., 156.,  ...,  44.,  27.,  16.],
           [229., 175., 148.,  ...,  27.,  35.,  27.],
           [214., 156., 157.,  ...,  28.,  22.,  28.],
           ...,
           [241., 245., 250.,  ...,  57., 101., 146.],
           [246., 250., 252.,  ...,  78., 105., 162.],
         

# ****MODEL FORMATION****

In [11]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.relu_stack = nn.Sequential(
            nn.Linear(2304, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 7),
            nn.ReLU(),
            nn.Linear(7, 7),
            nn.ReLU()
        )
        
    def forward(self, x):
        x  = self.flatten(x)
        output = self.relu_stack(x)
        return output
    
net = Net()

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-05)

In [13]:
def train(train_data, model, loss_fn, optimizer):
    size = len(train_data*7)
    count = 0
    for batch, (X, Y) in enumerate(train_data):
        pred = model(X)
        loss = loss_fn(pred, Y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch%700 == 0:
            output = f"loss:[{loss:0.2f}]   current:[{batch*len(X):5d}/{size}]"
            print(output)

In [14]:
def test(testdata, model, loss_fn, optimizer):
    size = len(testdata*7)
    
    loss, correct = 0, 0
    with torch.no_grad():
        for X, Y in testdata:
            pred = model(X)
            
            loss += loss_fn(pred, Y).item()
            correct += (pred.argmax(1)==Y).type(torch.float).sum().item()
            
        loss /= size
        correct /= size
            
        print(f"Avg loss{loss:0.2}  Accuracy:{correct*100:0.2f}%")
            

In [15]:
EPOCH = 20

for i in range(EPOCH):
    train(trainset, net, loss_fn, optimizer)
    test(testset, net, loss_fn, optimizer)
   

loss:[9.66]   current:[    0/28714]
loss:[2.02]   current:[ 4900/28714]
loss:[2.07]   current:[ 9800/28714]
loss:[1.92]   current:[14700/28714]
loss:[1.84]   current:[19600/28714]
loss:[1.97]   current:[24500/28714]
Avg loss0.27  Accuracy:25.53%
loss:[2.03]   current:[    0/28714]
loss:[1.94]   current:[ 4900/28714]
loss:[1.93]   current:[ 9800/28714]
loss:[1.88]   current:[14700/28714]
loss:[1.83]   current:[19600/28714]
loss:[1.97]   current:[24500/28714]
Avg loss0.27  Accuracy:25.21%
loss:[2.01]   current:[    0/28714]
loss:[1.94]   current:[ 4900/28714]
loss:[1.92]   current:[ 9800/28714]
loss:[1.92]   current:[14700/28714]
loss:[1.82]   current:[19600/28714]
loss:[1.97]   current:[24500/28714]
Avg loss0.27  Accuracy:26.20%
loss:[1.94]   current:[    0/28714]
loss:[1.76]   current:[ 4900/28714]
loss:[1.93]   current:[ 9800/28714]
loss:[1.87]   current:[14700/28714]
loss:[1.90]   current:[19600/28714]
loss:[1.97]   current:[24500/28714]
Avg loss0.27  Accuracy:26.11%
loss:[2.04]   cu

In [16]:
a = dataset[6000]
a

[tensor([[[251., 170., 177.,  ...,   0.,  55., 255.],
          [251., 157., 170.,  ...,   0.,  55., 255.],
          [251., 148., 149.,  ...,   0.,  54., 255.],
          ...,
          [253., 112.,  41.,  ...,   0.,  55., 255.],
          [254., 104.,  19.,  ...,   0.,  55., 255.],
          [255.,  80.,  15.,  ...,   0.,  55., 255.]]]),
 tensor(4)]

In [17]:
pred_output = net(a[0])

In [18]:
pred_output.argmax()

tensor(4)

In [19]:
torch.save(net, 'model.pth')